In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pandas as pd
from openpyxl import Workbook
from bs4 import BeautifulSoup
import re

#setam calea către driverul Chrome și deschidem browserul
PATH = "C:\\Users\\Larisa\\Desktop\\licenta\\chromedriver2" 
driver = webdriver.Chrome(PATH)

# deschide site-ul Blitz.ro și caută anunțuri de vânzare
url= "https://www.blitz.ro/cluj-napoca/vanzari-apartamente"
#url = "https://www.blitz.ro/cluj-napoca?utm_term=imobiliare%20cluj&utm_campaign=14484281179&utm_content=&utm_source=google&utm_medium=cpc&gclid=Cj0KCQiAjbagBhD3ARIsANRrqEtb1A5VEDDrbYpvgAd42_6u8Fc3HznDCHRUrpb0QIjVnquUbfhr59IaAr_gEALw_wcB"
driver.get(url)


anunturi = []
table = {}

count = 0
cards = driver.find_elements(By.CSS_SELECTOR, '.card')
for i in range(len(driver.find_elements(By.CSS_SELECTOR, '.card__content'))):
    try:
         # condiția de ieșire din buclă
        if count == 99:
            break
        # accesam linkul pentru fiecare anunț și extragem informațiile dorite din codul sursă
        card = driver.find_elements(By.CSS_SELECTOR, '.card__content')[i]
        link = card.find_element(By.CSS_SELECTOR, '.card__content--head a').get_attribute('href')
        print(link)
        driver.get(link)
        
        offer_features = driver.find_element(By.CSS_SELECTOR, '.offer-features')
        soup = BeautifulSoup(offer_features.get_attribute('innerHTML'), 'html.parser')
        
        offer_info = driver.find_element(By.CSS_SELECTOR, '.offer-informations')
        soup2 = BeautifulSoup(offer_info.get_attribute('innerHTML'), 'html.parser')
        
        anunt ={}

        # Find all the <strong> tags and use their text as column names
        for strong in soup.find_all('strong'):
            column_name = strong.text.strip()
            anunt[column_name] = ''

      #  offer_info = soup2.find('div', class_='offer-informations')
        if offer_info is not None:
            for item in soup2.find_all('div', class_='offer-informations--item'):
                strong_tags = item.find_all('strong')
                print(strong_tag)
                if strong_tags is not None:
                    for index, strong_tag in enumerate(strong_tags):
                        if 'Cartier : ' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                print(column_value)
                                anunt['Cartier'] = column_value
                        elif 'Pret/mp: ' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                numeric_value = re.search(r'\d+[\.,]?\d*', column_value)
                                if numeric_value:
                                    column_value = numeric_value.group(0).replace(',', '.')  # înlocuiți virgula cu punctul pentru valorile cu virgulă
                                print(column_value)
                                anunt['Pret/mp'] = column_value 
                        elif 'Actualizat:' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                anunt['Data anunt'] = column_value  
                            
        for li in soup.find_all('li'):
            for strong in li.find_all('strong'):
               
                column_name = strong.text.strip()
                column_value = ''
                if strong.next_sibling is not None:
                    column_value = strong.next_sibling.strip()
                numeric_value = re.search(r'\d+[\.,]?\d*', column_value)
                if numeric_value:
                    column_value = numeric_value.group(0).replace(',', '.')  # înlocuiți virgula cu punctul pentru valorile cu virgulă
                    print(column_value)    
                #table[column_name].append(column_value)
                anunt[column_name] = column_value

        print(anunt)        
        anunturi.append(anunt)
        
        
        driver.back()  # revenim la pagina de căutare pentru a continua să parcurgeți anunțuri
    except NoSuchElementException:
        continue
    count += 1 

df2 = pd.DataFrame(anunturi)

# Print the dataframe and export it to Excel
print(df2)
df2.to_excel('output.xlsx', index=False)
# inchidem browserul
driver.quit()    

C:\Users\Larisa\AppData\Local\Temp\ipykernel_3668\1414826448.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


KeyboardInterrupt: 

In [3]:
import pandas as pd
existing_data = pd.read_excel('output.xlsx')
recent_date = existing_data['Data anunt'].max()

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

from openpyxl import Workbook
from bs4 import BeautifulSoup
import re

#setam calea către driverul Chrome și deschidem browserul
PATH = "C:\\Users\\Larisa\\Desktop\\licenta\\chromedriver2" 
driver = webdriver.Chrome(PATH)

# deschide site-ul Blitz.ro și caută anunțuri de vânzare
url= "https://www.blitz.ro/cluj-napoca/vanzari-apartamente"
#url = "https://www.blitz.ro/cluj-napoca?utm_term=imobiliare%20cluj&utm_campaign=14484281179&utm_content=&utm_source=google&utm_medium=cpc&gclid=Cj0KCQiAjbagBhD3ARIsANRrqEtb1A5VEDDrbYpvgAd42_6u8Fc3HznDCHRUrpb0QIjVnquUbfhr59IaAr_gEALw_wcB"
driver.get(url)


anunturi = []
table = {}

count = 0
cards = driver.find_elements(By.CSS_SELECTOR, '.card')
for i in range(len(driver.find_elements(By.CSS_SELECTOR, '.card__content'))):
    try:
         # condiția de ieșire din buclă
        if count == 50:
            break
        # accesam linkul pentru fiecare anunț și extragem informațiile dorite din codul sursă
        card = driver.find_elements(By.CSS_SELECTOR, '.card__content')[i]
        link = card.find_element(By.CSS_SELECTOR, '.card__content--head a').get_attribute('href')
        print(link)
        driver.get(link)
        
        offer_features = driver.find_element(By.CSS_SELECTOR, '.offer-features')
        soup = BeautifulSoup(offer_features.get_attribute('innerHTML'), 'html.parser')
        
        offer_info = driver.find_element(By.CSS_SELECTOR, '.offer-informations')
        soup2 = BeautifulSoup(offer_info.get_attribute('innerHTML'), 'html.parser')
        
        anunt ={}

        # Find all the <strong> tags and use their text as column names
        for strong in soup.find_all('strong'):
            column_name = strong.text.strip()
            anunt[column_name] = ''

      #  offer_info = soup2.find('div', class_='offer-informations')
        if offer_info is not None:
            for item in soup2.find_all('div', class_='offer-informations--item'):
                strong_tags = item.find_all('strong')
                if strong_tags is not None:
                    for index, strong_tag in enumerate(strong_tags):
                        if 'Cartier : ' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                print(column_value)
                                anunt['Cartier'] = column_value
                        elif 'Pret/mp: ' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                numeric_value = re.search(r'\d+[\.,]?\d*', column_value)
                                if numeric_value:
                                    column_value = numeric_value.group(0).replace(',', '.')  # înlocuiți virgula cu punctul pentru valorile cu virgulă
                                print(column_value)
                                anunt['Pret/mp'] = column_value 
                        elif 'Actualizat:' in strong_tag.text:
                            column_value = ''
                            if strong_tag.next_sibling is not None:
                                column_value = strong_tag.next_sibling.strip()
                                anunt['Data anunt'] = column_value  
                            
        for li in soup.find_all('li'):
            for strong in li.find_all('strong'):
               
                column_name = strong.text.strip()
                column_value = ''
                if strong.next_sibling is not None:
                    column_value = strong.next_sibling.strip()
                numeric_value = re.search(r'\d+[\.,]?\d*', column_value)
                if numeric_value:
                    column_value = numeric_value.group(0).replace(',', '.')  # înlocuiți virgula cu punctul pentru valorile cu virgulă
                    print(column_value)    
                #table[column_name].append(column_value)
                anunt[column_name] = column_value

        print(anunt) 
        #anunt['Data anunt'] = column_value
        if pd.to_datetime(anunt['Data anunt']) > pd.to_datetime(recent_date):
            anunturi.append(anunt)
        
        
        driver.back()  # revenim la pagina de căutare pentru a continua să parcurgeți anunțuri
    except NoSuchElementException:
        continue
    count += 1 

df2 = pd.DataFrame(anunturi)
new_data = pd.DataFrame(anunturi)
df_final = pd.concat([existing_data, new_data], ignore_index=True)
# Print the dataframe and export it to Excel
print(df_final)
df_final.to_excel('output.xlsx', index=False)

# inchidem browserul
driver.quit()    

C:\Users\Larisa\AppData\Local\Temp\ipykernel_3668\2072393783.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


https://www.blitz.ro/cluj-napoca/apartament-vanzare-2-camere-baciu-blitz90473av
Baciu
1.350
2
51
1
1
1
1
1990
{'Nr. camere:': '2', 'Suprafață utilă': '51', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '1', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Nemobilat', 'Anul construcției:': '1990', 'Tip finisaj:': 'Semifinisat', 'Facilități:': '', 'Data anunt': '11.05.2023', 'Cartier': 'Baciu', 'Pret/mp': '1.350'}


C:\Users\Larisa\AppData\Local\Temp\ipykernel_3668\2072393783.py:95: UserWarning: Parsing '28.04.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  if pd.to_datetime(anunt['Data anunt']) > pd.to_datetime(recent_date):


https://www.blitz.ro/cluj-napoca/apartament-vanzare-2-camere-baciu-blitz90477av
Baciu
1.350
2
49
1
2
1
1990
{'Nr. camere:': '2', 'Suprafață utilă': '49', 'Nr. băi:': '1', 'Etaj:': '2', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Nemobilat', 'Anul construcției:': '1990', 'Tip finisaj:': 'Semifinisat', 'Facilități:': '', 'Data anunt': '11.05.2023', 'Cartier': 'Baciu', 'Pret/mp': '1.350'}
https://www.blitz.ro/cluj-napoca/apartament-vanzare-3-camere-baciu-blitz90479av
Baciu
1.350
3
52
1
1
2
1
1990
{'Nr. camere:': '3', 'Suprafață utilă': '52', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '2', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Nemobilat', 'Anul construcției:': '1990', 'Tip finisaj:': 'Semifinisat', 'Facilități:': '', 'Data 

https://www.blitz.ro/cluj-napoca/apartament-vanzare-2-camere-semicentral-blitz104814av
Semicentral
2.273
2
66
1
1
2
1
2022
{'Nr. camere:': '2', 'Suprafață utilă': '66', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '2', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Nemobilat', 'Anul construcției:': '2022', 'Tip finisaj:': 'Finisat', 'Materiale construcție:': 'BCA', 'Facilități:': '', 'Data anunt': '11.05.2023', 'Cartier': 'Semicentral', 'Pret/mp': '2.273'}
https://www.blitz.ro/cluj-napoca/apartament-vanzare-3-camere-baciu-blitz106804av
Baciu
1.797
3
59
1
1
3
2022
{'Nr. camere:': '3', 'Suprafață utilă': '59', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '3', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Mobilat/utilat', 'Anul construcției:': '2022', 'Tip fi

Floreşti
1.488
3
80
1
1
1
1
2015
{'Nr. camere:': '3', 'Suprafață utilă': '80', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '1', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Modern/lux', 'Anul construcției:': '2015', 'Tip finisaj:': 'Modern/lux', 'Materiale construcție:': 'Beton, Cărămidă', 'Modalitate vânzare:': 'Credit bancar, Cash', 'Facilități:': '', 'Data anunt': '11.05.2023', 'Cartier': 'Floreşti', 'Pret/mp': '1.488'}
https://www.blitz.ro/cluj-napoca/apartament-vanzare-3-camere-floresti-blitz115806av
Floreşti
1.417
3
60
1
1
1
1
2022
{'Nr. camere:': '3', 'Suprafață utilă': '60', 'Nr. băi:': '1', 'Nr. balcoane:': '1', 'Etaj:': '1', 'Locuri de parcare:': '1', 'Boxă la subsol:': 'Nu', 'Tip compartimentare:': 'Semidecomandat', 'Vechime apartament:': 'Nou', 'Tip imobil:': 'Bloc de apartamente', 'Dotări:': 'Mobilat/utilat', 'Anul construcției:': '2022', 'Ti